# Module 2: Drawing & Color Spaces

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/02_Drawing_and_Color.ipynb)

**Week 3-4: Drawing Primitives & Color Transformations**

## Learning Objectives
- Draw shapes, lines, and text on images
- Understand different color spaces (BGR, RGB, HSV, LAB)
- Convert between color spaces
- Apply thresholding techniques

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 2.1 Drawing Lines

In [ ]:
# Create blank canvas
canvas = np.zeros((400, 600, 3), dtype=np.uint8)

# Draw line: cv2.line(img, pt1, pt2, color, thickness)
cv2.line(canvas, (50, 50), (550, 50), (0, 255, 0), 2)  # Green horizontal
cv2.line(canvas, (50, 100), (550, 300), (255, 0, 0), 3)  # Blue diagonal
cv2.line(canvas, (300, 50), (300, 350), (0, 0, 255), 4)  # Red vertical

# Anti-aliased line
cv2.line(canvas, (50, 200), (550, 200), (255, 255, 0), 2, cv2.LINE_AA)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('Drawing Lines')
plt.axis('off')
plt.show()

## 2.2 Drawing Shapes

In [ ]:
canvas = np.zeros((500, 700, 3), dtype=np.uint8)

# Rectangle: cv2.rectangle(img, pt1, pt2, color, thickness)
cv2.rectangle(canvas, (50, 50), (200, 150), (0, 255, 0), 2)  # Outline
cv2.rectangle(canvas, (250, 50), (400, 150), (0, 255, 0), -1)  # Filled

# Circle: cv2.circle(img, center, radius, color, thickness)
cv2.circle(canvas, (100, 300), 50, (255, 0, 0), 2)  # Outline
cv2.circle(canvas, (300, 300), 50, (255, 0, 0), -1)  # Filled

# Ellipse: cv2.ellipse(img, center, axes, angle, startAngle, endAngle, color, thickness)
cv2.ellipse(canvas, (550, 100), (80, 40), 0, 0, 360, (0, 0, 255), 2)
cv2.ellipse(canvas, (550, 100), (80, 40), 45, 0, 180, (0, 255, 255), -1)

# Polygon
pts = np.array([[500, 250], [600, 250], [650, 350], [550, 400], [450, 350]], np.int32)
pts = pts.reshape((-1, 1, 2))
cv2.polylines(canvas, [pts], True, (255, 255, 0), 3)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('Drawing Shapes')
plt.axis('off')
plt.show()

## 2.3 Adding Text

In [ ]:
canvas = np.zeros((400, 800, 3), dtype=np.uint8)

# Different fonts
fonts = [
    (cv2.FONT_HERSHEY_SIMPLEX, 'SIMPLEX'),
    (cv2.FONT_HERSHEY_PLAIN, 'PLAIN'),
    (cv2.FONT_HERSHEY_DUPLEX, 'DUPLEX'),
    (cv2.FONT_HERSHEY_COMPLEX, 'COMPLEX'),
    (cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 'SCRIPT')
]

y_pos = 50
for font, name in fonts:
    cv2.putText(canvas, name, (50, y_pos), font, 1, (0, 255, 255), 2, cv2.LINE_AA)
    y_pos += 60

# Different sizes
cv2.putText(canvas, 'Small', (400, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv2.putText(canvas, 'Medium', (400, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(canvas, 'Large', (400, 300), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('Text Rendering')
plt.axis('off')
plt.show()

## 2.4 Color Spaces - BGR vs RGB vs HSV

In [ ]:
# Load sample image
!wget -q https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg -O sample.jpg
img = cv2.imread('sample.jpg')

# Convert to different color spaces
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Display
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original (BGR→RGB)')

axes[0, 1].imshow(hsv)
axes[0, 1].set_title('HSV')

axes[0, 2].imshow(lab)
axes[0, 2].set_title('LAB')

axes[1, 0].imshow(gray, cmap='gray')
axes[1, 0].set_title('Grayscale')

# Show individual HSV channels
h, s, v = cv2.split(hsv)
axes[1, 1].imshow(s, cmap='gray')
axes[1, 1].set_title('Saturation Channel')

axes[1, 2].imshow(v, cmap='gray')
axes[1, 2].set_title('Value Channel')

for ax in axes.flat:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2.5 Color Detection using HSV

In [ ]:
# Convert to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define color range (red in this example)
# HSV ranges: H: 0-179, S: 0-255, V: 0-255
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create masks
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask = mask1 + mask2

# Apply mask
result = cv2.bitwise_and(img, img, mask=mask)

# Display
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Red Color Mask')
axes[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
axes[2].set_title('Red Objects Only')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2.6 Thresholding

In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Simple threshold
_, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Otsu's thresholding
_, thresh2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Adaptive thresholding
thresh3 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                 cv2.THRESH_BINARY, 11, 2)
thresh4 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                 cv2.THRESH_BINARY, 11, 2)

# Display
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original Grayscale')

axes[0, 1].imshow(thresh1, cmap='gray')
axes[0, 1].set_title('Simple Threshold (127)')

axes[0, 2].imshow(thresh2, cmap='gray')
axes[0, 2].set_title("Otsu's Threshold")

axes[1, 0].imshow(thresh3, cmap='gray')
axes[1, 0].set_title('Adaptive Mean')

axes[1, 1].imshow(thresh4, cmap='gray')
axes[1, 1].set_title('Adaptive Gaussian')

axes[1, 2].axis('off')

for ax in axes.flat[:-1]:
    ax.axis('off')
plt.tight_layout()
plt.show()

## Project: Simple Drawing App

In [ ]:
# Create a simple drawing canvas
canvas = np.ones((600, 800, 3), dtype=np.uint8) * 255

# Add title
cv2.putText(canvas, 'My Digital Canvas', (250, 50), 
            cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 0), 3)

# Draw a sun
cv2.circle(canvas, (650, 100), 40, (0, 255, 255), -1)  # Yellow sun

# Draw a house
cv2.rectangle(canvas, (100, 300), (350, 500), (0, 100, 200), -1)  # House body
pts = np.array([[100, 300], [225, 200], [350, 300]], np.int32)
cv2.fillPoly(canvas, [pts], (0, 0, 200))  # Roof
cv2.rectangle(canvas, (180, 380), (270, 500), (100, 50, 0), -1)  # Door
cv2.circle(canvas, (250, 440), 5, (255, 255, 0), -1)  # Doorknob

# Draw a tree
cv2.rectangle(canvas, (480, 400), (520, 500), (0, 100, 50), -1)  # Trunk
cv2.circle(canvas, (500, 350), 70, (0, 200, 0), -1)  # Leaves

# Draw ground
cv2.rectangle(canvas, (0, 500), (800, 600), (0, 150, 0), -1)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('My Drawing')
plt.axis('off')
plt.show()

# Save the drawing
cv2.imwrite('my_drawing.png', canvas)
print("Drawing saved!")

## Summary

You learned:
- Drawing primitives (lines, circles, rectangles, polygons)
- Text rendering with different fonts
- Color space conversions (BGR, RGB, HSV, LAB)
- Color-based object detection using HSV
- Various thresholding techniques

**Next**: Module 3 - Filtering & Enhancement